In [1]:
import pandas as pd
import numpy as np
from ipynb.fs.defs.functions import *

In [19]:
table_2019 = '/Users/ethan/OneDrive - University of Surrey/Coursework/Final Year/FYP/Guardian_University_Guide_2019_.xlsx'
columns = ['satisfied with teaching (%)','satisfied with course (%)','Continuation','Expenditure per student / 10','Student:staff ratio','Career prospects (%)','Value added score/10','Entry Tariff','satisfied with feedback (%)']
df_2019 = pd.read_excel(table_2019, sheet_name='Institution', index_col='Institution')[columns]
df_2019.rename(columns={'satisfied with teaching (%)':'% Satisfied with Teaching','satisfied with course (%)':'% Satisfied with course','Expenditure per student / 10':'Expenditure per student (fte)','Student:staff ratio':'Student: staff ratio','Career prospects (%)':'Career prospects','Entry Tariff':'Average Entry Tariff','satisfied with feedback (%)':'% Satisfied with Assessment'},inplace=True)
table_2021 = '/Users/ethan/OneDrive - University of Surrey/Coursework/Final Year/FYP/Guardian_University_Guide_2021.xlsx'
columns = ['% Satisfied with Teaching','% Satisfied with course','Continuation','Expenditure per student (fte)','Student: staff ratio','Career prospects','Value added score/10','Average Entry Tariff','% Satisfied with Assessment']
df_2021 = pd.read_excel(table_2021, sheet_name='Institution', index_col='Name of Provider')[columns]
table_2020 = '/Users/ethan/OneDrive - University of Surrey/Coursework/Final Year/FYP/Guardian_University_Guide_2020.xlsx'
columns = ['% Satisfied with Teaching','% Satisfied with course','Continuation','Expenditure per student (fte)','Student: staff ratio','Career prospects','Value added score/10','Average Entry Tariff','% Satisfied with Assessment']
df_2020 = pd.read_excel(table_2020, sheet_name='Institution', index_col='Name of Provider')[columns]

In [3]:
combined = df_2021.combine_first(df_2020.combine_first(df_2019))
combined.head()

,% Satisfied with Teaching,% Satisfied with course,Continuation,Expenditure per student (fte),Student: staff ratio,Career prospects,Value added score/10,Average Entry Tariff,% Satisfied with Assessment
Aberdeen,86.1,87.1,94.9,4.4,16.2,82.0,6.6,183.0,69.5
Abertay Dundee,88.7,88.1,89.1,2.1,21.2,71.0,7.0,142.0,80.8
Aberystwyth,90.4,90.6,91.9,4.8,16.8,71.0,4.8,123.0,84.3
Anglia Ruskin,85.1,82.6,87.7,6.3,17.7,77.0,6.6,110.0,76.1
Aston,83.5,84.0,96.0,5.9,16.1,83.0,6.4,128.0,70.0


In [48]:
weights = {}
tot = 0
while True:
    try:
        for column in combined.columns:
            n = int(input(f'Enter weight for {column}'))
            weights[column] = float(n) / 100
            tot += n
            if tot > 100:
                raise Exception
            print(tot)
        if tot != 100:
            raise Exception
        break
    except TypeError:
        print('Enter an Integer')
    except Exception:
        print('Weights do not total 100, try again')
        
print(weights)

Enter weight for % Satisfied with Teaching10
10
Enter weight for % Satisfied with course5
15
Enter weight for Continuation10
25
Enter weight for Expenditure per student (fte)5
30
Enter weight for Student: staff ratio15
45
Enter weight for Career prospects15
60
Enter weight for Value added score/1015
75
Enter weight for Average Entry Tariff15
90
Enter weight for % Satisfied with Assessment10
100
{'% Satisfied with Teaching': 0.1, '% Satisfied with course': 0.05, 'Continuation': 0.1, 'Expenditure per student (fte)': 0.05, 'Student: staff ratio': 0.15, 'Career prospects': 0.15, 'Value added score/10': 0.15, 'Average Entry Tariff': 0.15, '% Satisfied with Assessment': 0.1}


In [49]:
final = combined.copy()
val_added = {
    10:1.8,
    9:1.5,
    8:0.95,
    7:0.5,
    6:0.15,
    5:-0.15,
    4:-0.5,
    3:-0.95,
    2:-1.5,
    1:-1.8
}

exclude = ["Expenditure per student (fte)", "Value added score/10"]
for column in combined.columns:
    if column in exclude:
        x = np.round(combined[column],0)
        final[column] = [val_added[int(i)] * weights[column] for i in x]
    else:
        final[column] = s_score(combined,column) * weights[column]
    if column == "Student: staff ratio":
        final[column] = -1 * final[column]

In [50]:
final.head()

,% Satisfied with Teaching,% Satisfied with course,Continuation,Expenditure per student (fte),Student: staff ratio,Career prospects,Value added score/10,Average Entry Tariff,% Satisfied with Assessment
Aberdeen,0.073246,0.053942,0.087214,-0.0250,-0.022355,0.088061,0.0750,0.273085,-0.063259
Abertay Dundee,0.164888,0.067316,-0.067335,-0.0750,-0.337614,-0.143775,0.0750,0.042289,0.191829
Aberystwyth,0.224808,0.100750,0.007275,-0.0075,-0.060186,-0.143775,-0.0225,-0.064665,0.270838
Anglia Ruskin,0.037999,-0.006239,-0.104640,0.0075,-0.116933,-0.017319,0.0750,-0.137844,0.085730
Aston,-0.018397,0.012484,0.116525,0.0075,-0.016050,0.109137,0.0225,-0.036519,-0.051972


In [51]:
final.insert(0,"Score",value=0)

In [58]:
x = []
for prov in final.index:
    x.append(sum(final.loc[prov,columns]))
x = [elem + abs(min(x)) for elem in x]
x = [elem * (100/max(x)) for elem in x]
final["Score"] = x
final.sort_values(by=["Score"]).head(20)


,Score,% Satisfied with Teaching,% Satisfied with course,Continuation,Expenditure per student (fte),Student: staff ratio,Career prospects,Value added score/10,Average Entry Tariff,% Satisfied with Assessment
Bedfordshire,0.000000,-0.060693,-0.091831,-0.333800,-0.0075,-0.243037,-0.080547,-0.0750,-0.188507,-0.022626
London Met,2.725259,-0.067742,-0.050373,-0.301824,0.0075,-0.123238,-0.249155,-0.0750,-0.216653,0.049611
Westminster,3.536518,-0.169959,-0.058397,0.001945,-0.0250,-0.318699,-0.185927,-0.0750,-0.087182,-0.085833
Glyndŵr,5.564546,0.030929,-0.027344,-0.192033,-0.0750,-0.210466,-0.294454,-0.0750,-0.174270,0.070640
Leeds Trinity,6.001251,-0.177008,-0.067758,-0.115299,-0.0250,-0.325004,-0.143775,0.0750,-0.171619,0.015750
Middlesex,6.835281,-0.229879,-0.085144,-0.070000,-0.0075,-0.072797,-0.207003,-0.0750,-0.109698,-0.054230
East London,7.772175,-0.071267,-0.051710,-0.064671,-0.0750,-0.243037,-0.249155,0.0225,-0.143473,-0.009081
Birkbeck,8.408451,0.096461,0.018403,-0.363111,-0.0250,0.002445,-0.118015,-0.1425,-0.236731,-0.098946
Goldsmiths,13.946655,-0.162909,-0.125265,-0.091317,-0.0075,0.059612,-0.185927,-0.0225,-0.042148,-0.133239
De Montfort,13.958277,-0.141761,-0.026300,-0.032695,-0.0475,-0.236731,-0.038395,-0.0225,-0.137844,-0.027141
